In [2]:
import pandas as pd
from pandas import Series, DataFrame
import codecs

In [3]:
#Video_idの被りを省く
with codecs.open("JPvideos.csv", "r", "UTF-8", "ignore") as file:
    df =  pd.read_csv(file)
    df.drop_duplicates(subset='video_id', keep='last', inplace=True)
#df

In [4]:
#必要のないと思われるCategoryID
remove_CategoryID = [3,4,5,6,7,8,9,11,12,13,14,16,28]
for n in remove_CategoryID:
    df = df[df["category_id"]!=n]
#df

In [5]:
#df_contains = df[df['title'].str.contains('l')]

In [6]:
#[]が含まれるなら1,含まれないなら0

buzz_word_Braces = ["【","】"]
for i in buzz_word_Braces:
    df.loc[df['title'].str.contains(i),"title_encode_Braces"] = 1
df.loc[df['title_encode_Braces'].isnull(),"title_encode_Braces"] = 0

In [7]:
#バズりやすい言葉があれば1,それ以外を0

buzz_word = ["神回","悲報","www","衝撃","危険","危機"]
for n in buzz_word:
    df.loc[df['title'].str.contains(n),"title_encode_word"] = 1
df.loc[df['title_encode_word'].isnull(),"title_encode_word"] = 0

In [8]:
#メッセージの長さ
df['len_title'] = df['title'].apply(lambda x: len(str(x).replace(' ', '')))
df["len_title"].describe()

count    12824.000000
mean        36.046241
std         17.125584
min          2.000000
25%         24.000000
50%         32.000000
75%         45.000000
max        100.000000
Name: len_title, dtype: float64

In [9]:
df.loc[df['len_title'] > 45,"title_encode_len"] = 2
df.loc[df['len_title'] < 45 ,"title_encode_len"] = 1
df.loc[df['len_title'] < 24,"title_encode_len"] = 0

In [10]:
#100万再生なら1,違うなら0とする
df.loc[df['views'] > 1000000,"view_encode"] = 1
df.loc[df['view_encode'].isnull(),"view_encode"] = 0
#df.head()

In [11]:
#データフレームを整える
df2 = df[['title_encode_Braces','title_encode_word','title_encode_len']]
df2 = df2.fillna(0)

In [12]:
#データの可視化
df_ex = df[['title_encode_Braces','title_encode_word','title_encode_len','view_encode']]
df_ex = df_ex.fillna(0)
df_ex.describe()

,title_encode_Braces,title_encode_word,title_encode_len,view_encode
count,12824.000000,12824.000000,12824.000000,12824.000000
mean,0.357923,0.024485,0.988771,0.041953
std,0.479408,0.154556,0.703174,0.200489
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,2.000000,1.000000


In [13]:
#モデルの構築
import numpy as np
import numpy.random as random
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

X = df2
Y = df['view_encode']
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [14]:
#決定木

from sklearn.tree import DecisionTreeClassifier

# 決定木インスタンス(木の深さ3)
model = DecisionTreeClassifier(max_depth=3)
#学習モデル構築。引数に訓練データの特徴量と、それに対応したラベル
model.fit(X_train, y_train)
predicted = model.predict(X_test)

# .scoreで正解率を算出。
print("train score:",model.score(X_train,y_train))
print("test score:",model.score(X_test,y_test))

train score: 0.9588271990018715
test score: 0.9557080474111042


In [15]:
#混合行列の作成
confusion_matrix(predicted,y_test)

array([[3064,  142],
       [   0,    0]])